# Firebase Firestore and Storage Configuration

In [1]:
# Firebase - Firestore Database Setup

import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from firebase_admin import storage

# Use a Service Account

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'storageBucket': 'suspicious-activity-database.appspot.com'})

# Real-Time Camera Detection

In [ ]:
# Imports

import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils
import firebase_admin
from firebase_admin import firestore
from datetime import datetime
from PIL import ImageGrab


run_once = 0

# datetime object containing current date and time

now = datetime.now()

# dd/mm/YY H:M:S

dt_string = now.strftime("Date: %B %d, %Y, Time: %H:%M:%S")

# Fucntion for calculating the mean distance

def loss_val(x1,x2):
    difference = x1-x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance/n_samples

    return mean_distance

model = load_model("model.h5")

cap = cv2.VideoCapture(0)

while (True):
    imagedump = []
    ret,frame = cap.read()

    # Resize the video file
    
    for i in range(10):
        ret,frame = cap.read()
        image = imutils.resize(frame,width=1000,height=1200)

        frame = cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray = 0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray = (gray-gray.mean())/gray.std()
        gray = np.clip(gray,0,1)
        imagedump.append(gray)
        
    # Set imagedump as a naumpy array

    imagedump = np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump = np.expand_dims(imagedump,axis=0)
    imagedump = np.expand_dims(imagedump,axis=4)

    output = model.predict(imagedump)

    loss = loss_val(imagedump,output) # final prediction value

    if frame.any() == None:
        print("none")


    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    if loss>0.00062: # Predict the loss value
        print('Suspicious Activity Detected')
        cv2.putText(image,"Suspicious Activity Detected",(60,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
        cv2.putText(image, dt_string, (120,700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2, cv2.LINE_AA)
        
        if run_once == 0:
            
            # save image to the local disk
            
            im = ImageGrab.grab()
            dt = datetime.now()
            fname = "snapshots/pic_{}.{}.png".format(dt.strftime("%H%M_%S"), dt.microsecond // 100000)
            im.save(fname, 'png') 
            
            # add snapshot to the cloud storage
            
            bucket = storage.bucket()
            blob = bucket.blob(fname)
            blob.upload_from_filename(fname)
            blob.make_public() # generate public url for snapshot
            
            
            # add suspicious activity details to the Firestore
            
            db = firestore.client()
            db.collection('suspicious_activities').add({'datetime':dt_string, 'image': blob.public_url})
            
            run_once = 1

    else:
        print('Normal Event')
        cv2.putText(image,"Normal Event",(240,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
        
    print(loss)    
        
    cv2.imshow("Suspicious Activity Detection System",image)

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 3s 3s/step
Normal Event
0.0005523979333285513
1/1 [==============================] - 0s 452ms/step
Normal Event
0.00055476354853407
1/1 [==============================] - 0s 448ms/step
Normal Event
0.0005459572112530138
1/1 [==============================] - 0s 423ms/step
Normal Event
0.0005298408952979479
1/1 [==============================] - 0s 454ms/step
Normal Event
0.0005051805905707211
1/1 [==============================] - 0s 460ms/step
Normal Event
0.0005048575800969819
1/1 [==============================] - 0s 484ms/step
Normal Event
0.0005043262551952506
1/1 [==============================] - 0s 481ms/step
Normal Event
0.0005116947764603877
1/1 [==============================] - 0s 448ms/step
Normal Event
0.0005184277875626516
1/1 [==============================] - 0s 465ms/step
Normal Event
0.0005563041827916184
1/1 [==============================] - 0s 440ms/step
Normal Event
0.0005024951397515752
1/1 [==============================] 

1/1 [==============================] - 0s 443ms/step
Normal Event
0.00039876686885794213
1/1 [==============================] - 0s 441ms/step
Normal Event
0.00039984922193044334
1/1 [==============================] - 0s 460ms/step
Normal Event
0.00039876760532365536
1/1 [==============================] - 0s 472ms/step
Normal Event
0.00040019366299240536
1/1 [==============================] - 0s 435ms/step
Normal Event
0.0003998466040144701
1/1 [==============================] - 0s 467ms/step
Normal Event
0.0003992621201755404
1/1 [==============================] - 0s 447ms/step
Normal Event
0.00039935400171912586
1/1 [==============================] - 0s 444ms/step
Normal Event
0.0003973741136306641
1/1 [==============================] - 0s 438ms/step
Normal Event
0.00039758969056463933
1/1 [==============================] - 0s 477ms/step
Normal Event
0.0003985077389987634
1/1 [==============================] - 0s 479ms/step
Normal Event
0.0003994528926271702
1/1 [=====================

1/1 [==============================] - 0s 478ms/step
Normal Event
0.00037536784271772876
1/1 [==============================] - 0s 471ms/step
Normal Event
0.00037085566209802865
1/1 [==============================] - 0s 435ms/step
Normal Event
0.00036967059215328686
1/1 [==============================] - 0s 438ms/step
Normal Event
0.0003685323799609463
1/1 [==============================] - 0s 494ms/step
Normal Event
0.0003684249142929264
1/1 [==============================] - 0s 461ms/step
Normal Event
0.000367196895712874
1/1 [==============================] - 0s 485ms/step
Normal Event
0.000370451657409576
1/1 [==============================] - 1s 515ms/step
Normal Event
0.00038019842211125756
1/1 [==============================] - 0s 471ms/step
Normal Event
0.0003797829261203684
1/1 [==============================] - 0s 438ms/step
Normal Event
0.0003821458884331477
1/1 [==============================] - 0s 446ms/step
Normal Event
0.0003810067665743548
1/1 [=========================

# Video Capture Detection

In [2]:
# Imports
import cv2
import numpy as np 
from keras.models import load_model
import argparse
from PIL import Image
import imutils
import firebase_admin
from firebase_admin import firestore
from datetime import datetime
from PIL import ImageGrab
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter.filedialog import askopenfile
from PIL import Image, ImageTk


run_once = 0

# datetime object containing current date and time

now = datetime.now()

# dd/mm/YY H:M:S

dt_string = now.strftime("Date: %B %d, %Y, Time: %H:%M:%S")

# Fucntion for calculating the mean distance

def loss_val(x1,x2):
    difference = x1-x2
    a,b,c,d,e = difference.shape
    n_samples = a*b*c*d*e
    sq_difference = difference**2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance/n_samples

    return mean_distance

model = load_model("model.h5")

# GUI for file selection

root = tk.Tk()

root.title('Suspicious Activity Detection System')

canvas = tk.Canvas(root, width = 600, height = 600)
canvas.grid(columnspan = 5, rowspan = 5)

#logo

logo = Image.open('icon/logo.png')
logo = ImageTk.PhotoImage(logo)
logo_label = tk.Label(image = logo)
logo_label.image = logo
logo_label.grid(column = 2, row = 0)

lbl = tk.Label(root, text = "Suspicious Activity Detection System", font="Raleway")
lbl.grid(columnspan = 5, column = 0, row = 1)
lbl.grid(row = 1, column = 0)

btn = tk.Button(root, text = 'Browse', command = lambda:browse_file(), 
                font="Raleway", bg="#20bebe", fg="white", height = 2, width = 15)
btn.grid(row = 2, column = 2) 

lbl1 = tk.Label(root, text = "Select the File by Clicking Broswers Button", font="Raleway")
lbl1.grid(columnspan = 5, column = 0, row = 1)
lbl1.grid(row = 3, column = 0)

# Function for browse video file and get the path

def browse_file():
    filename = filedialog.askopenfilename()
    global cap
    cap = cv2.VideoCapture(filename)
    print(cap.isOpened())
    print(filename)
    root.destroy()

root.mainloop()  # Keep the window open

while (True):
    imagedump = []
    ret,frame = cap.read()

    # Resize the video file
    
    for i in range(10):
        ret,frame = cap.read()
        image = imutils.resize(frame,width=1000,height=1200)

        frame = cv2.resize(frame, (227,227), interpolation = cv2.INTER_AREA)
        gray = 0.2989*frame[:,:,0]+0.5870*frame[:,:,1]+0.1140*frame[:,:,2]
        gray = (gray-gray.mean())/gray.std()
        gray = np.clip(gray,0,1)
        imagedump.append(gray)

    # Set imagedump as a naumpy array
        
    imagedump = np.array(imagedump)

    imagedump.resize(227,227,10)
    imagedump = np.expand_dims(imagedump,axis=0)
    imagedump = np.expand_dims(imagedump,axis=4)

    output = model.predict(imagedump)

    loss = loss_val(imagedump,output) # final prediction value

    if frame.any() == None:
        print("none")


    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    if loss>0.00060: # Predict the loss value
        print('Suspicious Activity Detected')
        cv2.putText(image,"Suspicious Activity Detected",(60,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
        cv2.putText(image, dt_string, (120,700), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,255), 2, cv2.LINE_AA)
        
        if run_once == 0:
            
            # save image to the local disk
            
            im = ImageGrab.grab()
            dt = datetime.now()
            fname = "snapshots/pic_{}.{}.png".format(dt.strftime("%H%M_%S"), dt.microsecond // 100000)
            im.save(fname, 'png') 
            
            # add snapshot to the cloud storage
            
            bucket = storage.bucket()
            blob = bucket.blob(fname)
            blob.upload_from_filename(fname)
            blob.make_public() # generate public url for snapshot
            
            
            # add suspicious activity details to the Firestore
            
            db = firestore.client()
            db.collection('suspicious_activities').add({'datetime':dt_string, 'image': blob.public_url})
            
            run_once = 1

    else:
        print('Normal Event')
        cv2.putText(image,"Normal Event",(240,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)
        
    print(loss)    
        
    cv2.imshow("Suspicious Activity Detection System",image)

cap.release()
cv2.destroyAllWindows()

True
C:/Users/User/Documents/Suspicious Activity Detection System/System/1.mp4
1/1 [==============================] - 1s 743ms/step
Normal Event
0.0005334739960399403
1/1 [==============================] - 0s 208ms/step
Normal Event
0.0005342776704870461
1/1 [==============================] - 0s 201ms/step
Normal Event
0.0005353051762048452
1/1 [==============================] - 0s 198ms/step
Normal Event
0.0005374172631325617
1/1 [==============================] - 0s 211ms/step
Normal Event
0.0005392903470617757
1/1 [==============================] - 0s 203ms/step
Normal Event
0.0005405257056757106
1/1 [==============================] - 0s 199ms/step
Normal Event
0.0005418104510395315
1/1 [==============================] - 0s 202ms/step
Normal Event
0.0005437442717670955
1/1 [==============================] - 0s 198ms/step
Normal Event
0.0005438497892556834
1/1 [==============================] - 0s 211ms/step
Normal Event
0.0005435225940299775
1/1 [==============================] - 0s

1/1 [==============================] - 0s 199ms/step
Suspicious Activity Detected
0.0006333127424336112


AttributeError: 'NoneType' object has no attribute 'shape'